In [1]:
from Embed import *
import pandas as pd
import numpy as np
import random
import networkx as nx
from tqdm import tqdm
import re
import matplotlib.pyplot as plt
import collections

from sklearn import svm
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, roc_auc_score, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix


Bad key "text.kerning_factor" on line 4 in
C:\Users\tarik\Anaconda3\envs\graphs\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
http://github.com/matplotlib/matplotlib/blob/master/matplotlibrc.template
or from the matplotlib source distribution


In [ ]:
def collectAdarScores(G, train_edges_name):
    
    # find which edges are unconnected in the training
    df_train = pd.read_csv(train_edges_name)
    df_train = df_train.replace(np.nan, 'nan', regex=True)
    #print(G.neighbors('nan'))
    #print(err)
    #list_unconnected = df_train.index[df_train['training_labels'] == 0].tolist() #df_train.where(df_train['training_labels']==0))

    list_real_labels = []
    list_pred_scores = []
    count =0
    for i_row in range(len(df_train.node1)): # for each training set data
        node1 = df_train.node1[i_row]
        node2 = df_train.node2[i_row]
        print(count)
        count = count + 1
        if(node1=='nan' or node2=='nan'):
            print('node1 :   '+str(node1)+'           node2 :'+str(node2))
            continue     
        # Find all nbrs of node1 and node2 in training graph that overlap
        list_nbrs = sorted(nx.common_neighbors(G, node1, node2))
            
        total_sum = 0
        # if list_nbrs isn't empty, find the weights of all the edges connected to the nbrs
        for i in range(len(list_nbrs)):
            curr_weight = G.degree(list_nbrs[i], weight='weight')
            total_sum += -1/np.log(curr_weight)

        #
        list_real_labels.append(df_train.labels[i_row])
        list_pred_scores.append(total_sum)

    return list_pred_scores, list_real_labels


In [ ]:
model = GaussianNB()

# Train the model using the training sets
model.fit(np.array(list_pred_scores_train).reshape(-1, 1),list_real_labels_train)